In [1]:
from os import getcwd, path
import sys
import matplotlib.pyplot as plt
import numpy as np

BASE_PATH = path.dirname(getcwd())
sys.path.append(BASE_PATH)

from text_classification.convnet.train import trainIters

In [2]:
from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

/Users/2359media/anaconda/envs/botbot-nlp/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Transform the data back to plain text

In [3]:
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+3) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}
# We actually don't need these tokens
id_to_word[0] = ""
id_to_word[1] = ""
id_to_word[2] = ""

def inverse_transform(seq):
    try:
        return ' '.join(id_to_word[id] for id in seq).strip()
    except:
        print(seq)
        return seq

In [ ]:
x_train_text = [inverse_transform(item) for item in x_train]
x_test_text  = [inverse_transform(item) for item in x_test]

In [ ]:
training_data = [(item, y_train[idx]) for idx, item in enumerate(x_train_text)]
classes = ['positive', 'negative']

losses, model = trainIters(training_data, 
                           classes, 
                           n_iters=10, 
                           log_every=1, 
                           verbose=1,
                           learning_rate=1e-3, 
                           batch_size=64)

Importing /Users/2359media/Documents/botbot-nlp/data/fasttext/crawl-300d-2M.vec...


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
fig, ax = plt.subplots()
loc = ticker.MultipleLocator(base=5)
ax.yaxis.set_major_locator(loc)
plt.plot(losses)

In [ ]:
from text_classification.convnet.train import evaluate_all
test_data = [(item, y_test[idx]) for idx, item in enumerate(x_test_text)]
print(evaluate_all(model, test_data))

In [ ]:
import torch.nn.functional as F
from common.utils import argmax, to_scalar
from text_classification.fast_text.model import process_sentences

TEST_SENTENCE = x_train_text[5]
print(TEST_SENTENCE)

result = model(*process_sentences([TEST_SENTENCE]))
result = F.softmax(result, dim=1)
max_idx = argmax(result)
print((classes[max_idx], to_scalar(result[0][max_idx])))